In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv")

In [2]:
import pandas as pd

unique_pizza_ids = df['pizza_name_id'].unique()

dataframes = {}

for pizza_id in unique_pizza_ids:
    df_filtered = df[df['pizza_name_id'] == pizza_id][['week_number', 'quantity', 'order_date']]
    dataframes[pizza_id] = df_filtered

In [3]:
example_pizza_id = unique_pizza_ids[89]
df = dataframes[example_pizza_id]
df

,week_number,quantity,order_date
617,2,1,2015-01-05
871,2,1,2015-01-07
969,2,1,2015-01-07
972,2,1,2015-01-07
1247,2,1,2015-01-09
...,...,...,...
47720,52,1,2015-12-22
47753,52,1,2015-12-22
48282,53,1,2015-12-28
48356,53,1,2015-12-29


In [4]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

results = {}

for j in range(0, 91):
    example_pizza_id = unique_pizza_ids[j]
    df = dataframes[example_pizza_id]
    
    # Group by 'week_number' and sum 'quantity'
    df_weekly = df.groupby('week_number').sum().reset_index()
    df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)
    
    # Create weekly date range starting from min order date
    df_weekly['ds'] = pd.date_range(start=df['order_date'].min(), periods=len(df_weekly), freq='W')

    # Split the data into training (70%) and test (30%)
    split_point = int(len(df_weekly) * 0.7)
    train_df = df_weekly[:split_point]
    test_df = df_weekly[split_point:]

    # Fit the Prophet model
    model = Prophet()
    model.fit(train_df)

    # Create future dataframe for test set and predict
    future = model.make_future_dataframe(periods=len(test_df), freq='W')
    forecast = model.predict(future)

    # Extract forecast for the test set
    test_forecast = forecast.iloc[split_point:split_point + len(test_df)][['ds', 'yhat']]

    # Calculate MAPE
    mape = mean_absolute_percentage_error(test_df['y'], test_forecast['yhat'])

    # Get next week forecast and round it to the nearest integer
    week_forecast = round(forecast.tail(1)['yhat'].values[0])

    # Store the results in the dictionary
    results[f'unique_pizza_ids[{j}]'] = {
        'Pizza_id': unique_pizza_ids[j],
        'MAPE': mape,
        'Next_Week_Prediction': week_forecast
    }

# Results can be accessed via the 'results' dictionary


c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
21:38:17 - cmdstanpy - INFO - Chain [1] start processing
21:38:17 - cmdstanpy - INFO - Chain [1] done processing
21:38:19 - cmdstanpy - INFO - Chain [1] start processing
21:38:19 - cmdstanpy - INFO - Chain [1] done processing
21:38:20 - cmdstanpy - INFO - Chain [1] start processing
21:38:20 - cmdstanpy - INFO - Chain [1] done processing
21:38:21 - cmdstanpy - INFO - Chain [1] start processing
21:38:21 - cmdstanpy - INFO - Chain [1] done processing
21:38:22 - cmdstanpy - INFO - Chain [1] start processing
21:38:22 - cmdstanpy - INFO - Chain [1] done processing
21:38:23 - cmdstanpy - INFO - Chain [1] start processing
21:38:24 - cmdstanpy - INFO - 

In [27]:
results

{'unique_pizza_ids[0]': {'Pizza_id': 'hawaiian_m',
  'MAPE': 0.8439071111900813,
  'Next_Week_Prediction': 12},
 'unique_pizza_ids[1]': {'Pizza_id': 'classic_dlx_m',
  'MAPE': 0.33170160310669905,
  'Next_Week_Prediction': 28},
 'unique_pizza_ids[2]': {'Pizza_id': 'five_cheese_l',
  'MAPE': 0.40800470502298936,
  'Next_Week_Prediction': 29},
 'unique_pizza_ids[3]': {'Pizza_id': 'ital_supr_l',
  'MAPE': 0.4404441090967258,
  'Next_Week_Prediction': 16},
 'unique_pizza_ids[4]': {'Pizza_id': 'mexicana_m',
  'MAPE': 1.0282360700544684,
  'Next_Week_Prediction': 10},
 'unique_pizza_ids[5]': {'Pizza_id': 'thai_ckn_l',
  'MAPE': 0.2833175934456438,
  'Next_Week_Prediction': 31},
 'unique_pizza_ids[6]': {'Pizza_id': 'ital_supr_m',
  'MAPE': 0.42580351115161086,
  'Next_Week_Prediction': 19},
 'unique_pizza_ids[7]': {'Pizza_id': 'prsc_argla_l',
  'MAPE': 0.45252481344475387,
  'Next_Week_Prediction': 6},
 'unique_pizza_ids[8]': {'Pizza_id': 'bbq_ckn_s',
  'MAPE': 0.6163155502533837,
  'Next_Wee

In [5]:
import pandas as pd


# Extract the values from the dictionary and put them into a list
values = [v for v in results.values()]

# Create the DataFrame
df = pd.DataFrame(values)

print(df)


         Pizza_id      MAPE  Next_Week_Prediction
0      hawaiian_m  0.843907                    12
1   classic_dlx_m  0.331702                    28
2   five_cheese_l  0.408005                    29
3     ital_supr_l  0.440444                    16
4      mexicana_m  1.028236                    10
..            ...       ...                   ...
86  ckn_alfredo_l  0.718917                     3
87    calabrese_s  0.637664                     2
88    the_greek_l  0.956909                     7
89  soppressata_m  0.514595                     5
90    calabrese_l  0.401525                     5

[91 rows x 3 columns]


In [29]:
values

[{'Pizza_id': 'hawaiian_m',
  'MAPE': 0.8439071111900813,
  'Next_Week_Prediction': 12},
 {'Pizza_id': 'classic_dlx_m',
  'MAPE': 0.33170160310669905,
  'Next_Week_Prediction': 28},
 {'Pizza_id': 'five_cheese_l',
  'MAPE': 0.40800470502298936,
  'Next_Week_Prediction': 29},
 {'Pizza_id': 'ital_supr_l',
  'MAPE': 0.4404441090967258,
  'Next_Week_Prediction': 16},
 {'Pizza_id': 'mexicana_m',
  'MAPE': 1.0282360700544684,
  'Next_Week_Prediction': 10},
 {'Pizza_id': 'thai_ckn_l',
  'MAPE': 0.2833175934456438,
  'Next_Week_Prediction': 31},
 {'Pizza_id': 'ital_supr_m',
  'MAPE': 0.42580351115161086,
  'Next_Week_Prediction': 19},
 {'Pizza_id': 'prsc_argla_l',
  'MAPE': 0.45252481344475387,
  'Next_Week_Prediction': 6},
 {'Pizza_id': 'bbq_ckn_s',
  'MAPE': 0.6163155502533837,
  'Next_Week_Prediction': 11},
 {'Pizza_id': 'the_greek_s',
  'MAPE': 0.521366338752449,
  'Next_Week_Prediction': 5},
 {'Pizza_id': 'spinach_supr_s',
  'MAPE': 0.6160515672856844,
  'Next_Week_Prediction': 9},
 {'Pizz

In [6]:
idf = pd.read_csv('Ingre.csv')
idf

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...,...
513,veggie_veg_s,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,veggie_veg_s,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,veggie_veg_s,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,veggie_veg_s,The Vegetables + Vegetables Pizza,Spinach,15.0


In [7]:
unique_ingredients = idf['pizza_ingredients'].unique()

ingredients_dict = {ingredient: 0 for ingredient in unique_ingredients}

(ingredients_dict)

{'Barbecued Chicken': 0,
 'Red Peppers': 0,
 'Green Peppers': 0,
 'Tomatoes': 0,
 'Red Onions': 0,
 'Barbecue Sauce': 0,
 'Bacon': 0,
 'Pepperoni': 0,
 'Italian Sausage': 0,
 'Chorizo Sausage': 0,
 'Brie Carre Cheese': 0,
 'Prosciutto': 0,
 'Caramelized Onions': 0,
 'Pears': 0,
 'Thyme': 0,
 'Garlic': 0,
 '?duja Salami': 0,
 'Pancetta': 0,
 'Friggitello Peppers': 0,
 'Chicken': 0,
 'Artichokes': 0,
 'Spinach': 0,
 'Jalapeno Peppers': 0,
 'Fontina Cheese': 0,
 'Gouda Cheese': 0,
 'Mushrooms': 0,
 'Asiago Cheese': 0,
 'Alfredo Sauce': 0,
 'Pesto Sauce': 0,
 'Mozzarella Cheese': 0,
 'Provolone Cheese': 0,
 'Smoked Gouda Cheese': 0,
 'Romano Cheese': 0,
 'Blue Cheese': 0,
 'Ricotta Cheese': 0,
 'Gorgonzola Piccante Cheese': 0,
 'Parmigiano Reggiano Cheese': 0,
 'Green Olives': 0,
 'Feta Cheese': 0,
 'Sliced Ham': 0,
 'Pineapple': 0,
 'Capocollo': 0,
 'Goat Cheese': 0,
 'Oregano': 0,
 'Calabrese Salami': 0,
 'Eggplant': 0,
 'Zucchini': 0,
 'Kalamata Olives': 0,
 'Sun-dried Tomatoes': 0,
 'P

In [32]:
len(ingredients_dict)

64

In [8]:
df1uni = df['Pizza_id'].unique()

In [9]:
for i in range(0,91):
    filtered_rows = idf[idf['pizza_name_id'] == df1uni[i]]
    prediction = df.loc[df['Pizza_id'] ==df1uni[i], 'Next_Week_Prediction'].values[0]
    filtered_df = idf[idf['pizza_name_id'] == df1uni[i]].copy()
    filtered_df['Total_Qty_Needed'] = filtered_df['Items_Qty_In_Grams'] * prediction
    for index, row in filtered_df.iterrows():
        ingredient = row['pizza_ingredients']
        total_qty_needed = row['Total_Qty_Needed']
        if ingredient in ingredients_dict:
            ingredients_dict[ingredient] = total_qty_needed

In [10]:
ingredients_dict

{'Barbecued Chicken': 570.0,
 'Red Peppers': 45.0,
 'Green Peppers': 180.0,
 'Tomatoes': 150.0,
 'Red Onions': 300.0,
 'Barbecue Sauce': 190.0,
 'Bacon': 1200.0,
 'Pepperoni': 360.0,
 'Italian Sausage': 400.0,
 'Chorizo Sausage': 2000.0,
 'Brie Carre Cheese': 360.0,
 'Prosciutto': 360.0,
 'Caramelized Onions': 300.0,
 'Pears': 120.0,
 'Thyme': 60.0,
 'Garlic': 75.0,
 '?duja Salami': 300.0,
 'Pancetta': 450.0,
 'Friggitello Peppers': 75.0,
 'Chicken': 450.0,
 'Artichokes': 105.0,
 'Spinach': 210.0,
 'Jalapeno Peppers': 120.0,
 'Fontina Cheese': 100.0,
 'Gouda Cheese': 120.0,
 'Mushrooms': 200.0,
 'Asiago Cheese': 90.0,
 'Alfredo Sauce': 60.0,
 'Pesto Sauce': 140.0,
 'Mozzarella Cheese': 100.0,
 'Provolone Cheese': 870.0,
 'Smoked Gouda Cheese': 870.0,
 'Romano Cheese': 870.0,
 'Blue Cheese': 870.0,
 'Ricotta Cheese': 1200.0,
 'Gorgonzola Piccante Cheese': 960.0,
 'Parmigiano Reggiano Cheese': 2160.0,
 'Green Olives': 110.0,
 'Feta Cheese': 280.0,
 'Sliced Ham': 1140.0,
 'Pineapple': 855

In [11]:
df_ingredients = pd.DataFrame(list(ingredients_dict.items()), columns=['Ingredient', 'Total_Qty_Needed'])

print(df_ingredients)

                 Ingredient  Total_Qty_Needed
0         Barbecued Chicken             570.0
1               Red Peppers              45.0
2             Green Peppers             180.0
3                  Tomatoes             150.0
4                Red Onions             300.0
..                      ...               ...
59                   Onions             120.0
60       Soppressata Salami             500.0
61        Peperoncini verdi              90.0
62  Thai Sweet Chilli Sauce             160.0
63         Beef Chuck Roast             700.0

[64 rows x 2 columns]


In [12]:
df_ingredients.to_excel('final_list.xlsx',index =False)